Before we begin, we will change a few settings to make the notebook look a bit prettier

In [1]:
%%html
<style> body {font-family: "Calibri", cursive, sans-serif;} </style>

<img src="https://github.com/IKNL/guidelines/blob/master/resources/logos/iknl_nl.png?raw=true" width=200 align="right">

# 00 - DeepSurv Using Keras
Originally, DeepSurv is implemented in Theano (using Lasagne).
However, [Theano is no longer supported](https://groups.google.com/forum/#!msg/theano-users/7Poq8BZutbY/rNCIfvAEAwAJ).
Therefore, I thought it would be a good idea to do my own implementation
of it using Keras. Not only I think this will give increase DeepSurv's use,
but it is also a good learning exercise for myself. 

## Preliminaries

Import packages

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ActivityRegularization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.regularizers import l2

import lifelines
from lifelines import datasets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import h5py

import logzero
from logzero import logger

In [ ]:
logzero.logfile("./logfile.log", maxBytes=1e6, backupCount=2)

## Get data
We will use the [Rotterdam & German Breast Cancer Study Group (GBSG) dataset](https://ascopubs.org/doi/abs/10.1200/jco.1994.12.10.2086).
The dataset was fetched from the [czifan's `DeepSurv.pytorch` repository](https://github.com/czifan/DeepSurv.pytorch)
(an attempt to implement DeepSurv in yet another platform, in this case
PyTorch).

In [ ]:
rossi_dataset = datasets.load_rossi()
E = np.array(rossi_dataset["arrest"])
Y = np.array(rossi_dataset["week"])
X = np.array(rossi_dataset)
X = X.astype('float64')
X = X[:,2:]

n_patients = X.shape[0]
n_features = X.shape[1]

In [ ]:
logger.info("Partitioning data...")
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=0)
X_train, X_val, E_train, E_val = train_test_split(X, E, test_size=0.25, random_state=0)
logger.info("DONE!")


#Standardize
scaler = StandardScaler().fit(X_train[:,[1,6]])

X_train[:, [1,6]] = scaler.transform(X_train[:, [1,6]])
X_val[:, [1,6]] = scaler.transform(X_val[:, [1,6]])

#Sorting for NNL!
sort_idx = np.argsort(Y_train)[::-1]
X_train = X_train[sort_idx]
Y_train = Y_train[sort_idx]
E_train = E_train[sort_idx]

n_patients_train = X_train.shape[0]

In [ ]:
#Loss Function
def negative_log_likelihood(E):
    def loss(y_true, y_pred):
        # print("Y true")
        # print(y_true)
        # tf.print(y_true)
        
        print("Y pred")
        tf.print(y_pred)
        print(tf.shape(y_pred))
        
        hazard_ratio = tf.math.exp(y_pred)
        # print("Hazard ratio")
        # tf.print(hazard_ratio)
        
        # print("log risk")
        log_risk = tf.math.log(tf.math.cumsum(hazard_ratio))
        # tf.print(log_risk)
        
        # print("uncensored likelihood")
        uncensored_likelihood = tf.transpose(y_pred) - log_risk
        # tf.print(uncensored_likelihood)
        
        # print("censored likelihood")
        censored_likelihood = uncensored_likelihood * E
        # tf.print(censored_likelihood)
        
        # print("num observed events")
        num_observed_events = tf.math.cumsum(E)
        num_observed_events = tf.cast(num_observed_events, dtype=tf.float32)
        # tf.print(num_observed_events)
        
        num_observed_events = tf.constant(1, dtype=tf.float32)
        
        neg_likelihood_ = -tf.math.reduce_sum(censored_likelihood)
        neg_likelihood = neg_likelihood_ / num_observed_events
        # tf.print(neg_likelihood)
        
        return neg_likelihood
    
    return loss

In [ ]:
#Keras model
model = Sequential()
model.add(Dense(units=32, activation='relu', kernel_initializer='glorot_uniform', input_shape=(n_features,))) # shape = length, dimension
model.add(Dropout(0.5))
model.add(Dense(units=32, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation="linear", kernel_initializer='glorot_uniform', kernel_regularizer=l2(0.01)))
model.add(ActivityRegularization(l2=0.01))

sgd = SGD(lr=1e-5, decay=0.01, momentum=0.9, nesterov=True)
rmsprop = RMSprop(lr=1e-5, rho=0.9, epsilon=1e-8)
model.compile(loss=negative_log_likelihood(E_train), optimizer=sgd)

model.summary()

####
print('Training...')
model.fit(X_train, Y_train, batch_size=n_patients_train, epochs=100, shuffle=False)  # Shuffle False --> Important!!

In [ ]:
hr_pred = model.predict(X_train)
hr_pred = np.exp(hr_pred)
ci = lifelines.utils.concordance_index(Y_train, -hr_pred, E_train)

hr_pred2 = model.predict(X_val)
hr_pred2 = np.exp(hr_pred2)
ci2 = lifelines.utils.concordance_index(Y_val, -hr_pred2, E_val)

print(f"Concordance Index for training dataset: {ci}")
print(f"Concordance Index for test dataset: {ci2}")

In [ ]:
# Cox Fitting
cph = lifelines.CoxPHFitter()
cph.fit(rossi_dataset, 'week', event_col='arrest')

cph.print_summary(decimals=3, style='ascii')  # access the results using cf.summary